In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import logging

logging.basicConfig(filename='log.log', filemode='a', format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
pd.set_option('display.max_columns', None)


In [ ]:
def popZeroID(row):
    x = row['TitleID'].split(".")
    return(x[0])

def popZeroDate(row):
    x = row['YearMonth'].split(".")
    return(x[0])

def splitYear(row):
    year = row['YearMonth'][:4]
    month = row['YearMonth'][4:6]
    monthyear = '{}/{}'.format(month, year)
    return(monthyear)


In [ ]:

# Read in new Print and ER sales
PRH_new = pd.read_excel(open('data_update/{}.xlsx'.format(new_pe_data), 'rb'),
              sheet_name='PRH_Jan-Feb') 

# Read in new EL sales
thirdPartyTransacations_new = pd.read_excel(open('data_update/{}'.format(new_el_file), 'rb'),
              sheet_name='3rdPartyEbookUpdates') 


In [ ]:
# # Convert Title ID and ISBN to string for existing data
existing_data['TitleID'] = existing_data['TitleID'].astype(str)
existing_data['ISBN'] = existing_data['ISBN'].astype(str)


In [ ]:
# Combine all new sales into a single DF
new_data = pd.concat([PRH_new, thirdPartyTransacations_new])

# We need to remove Gratis copies from the sales data
new_data_clean = (new_data[ 'revenuetype' ] != 'Gratis copies')
new_data = new_data[new_data_clean]

# Convert date into date format
# Drop any empty rows, convert 'TitleID' and 'PRINT isbn' to a string.
# Group everything by 'ISBN', 'TitleID', and 'Date (month)'
# Rename Print isbn as just 'isbn', which we need to do in order to merge print and electronic sales
new_data.rename(columns={'date': 'Date', 'unitssold': 'Units Sold', 'saleprice': 'Dom. Sale Price', 'actualamount': 'Amount Sold', 'isbn': 'ISBN'}, inplace=True)

new_data['Date'] = pd.to_datetime(new_data.Date)
new_data['Date'] = new_data['Date'].dt.strftime('%m/%Y')

new_data.dropna(how="all", inplace=True)
new_data['TitleID'] = new_data['TitleID'].astype(str)
new_data['ISBN'] = new_data['ISBN'].astype(str)



In [ ]:
# Divide up print and e sales
print_sales = new_data[new_data['revenuetype'] != 'Electronic']
print_sales = print_sales.groupby(['TitleID', 'ISBN', 'Date']).sum()
print_sales.reset_index(inplace=True)
print_sales['Format'] = 'Print'
print_sales

er_sales = new_data[new_data['revenuetype'] == 'Electronic']
er_sales = er_sales.groupby(['TitleID', 'ISBN', 'Date']).sum()
er_sales.reset_index(inplace=True)
er_sales['TitleID'] = er_sales.apply(popZeroID, axis=1)
er_sales['Format'] = 'Electronic'

In [ ]:
# Merge print and electronic sales
merged_sales = pd.concat([existing_data,er_sales,print_sales])
merged_sales = merged_sales[['TitleID', 'ISBN', 'Date', 'Units Sold', 'Amount Sold', 'Format']]

In [ ]:
s22 = pd.read_csv('data_input/{}'.format(season_list_file))
s22.dropna(subset=['IDTitle'], inplace=True)
s22['IDTitle'] = s22['IDTitle'].astype('str')
s22_tid = s22['IDTitle'].to_list()
# Filter By S22 and F22 titles only
merged_sales = merged_sales[merged_sales['TitleID'].isin(s22_tid)]

In [ ]:
merged_sales.to_csv('data_update/merged_sales_update.csv', index=None)